In [1]:
import numpy as np
import h5py
import pandas as pd

In [2]:
%pwd

'/xdisk/gbesla/katiechambe/pears/notebooks'

In [69]:
old135D = pd.read_csv("dwarfPairsDark/preisolation/data/raw/dwarfGroups_135.csv")

print("Old number of groups -- dwarf dark:",len(old135D))


Old number of groups -- dwarf dark: 13083


In [16]:
old135DD = pd.read_csv("dwarfPairsDark/data/raw/dwarfGroups_135.csv")

old135D = pd.read_csv("dwarfPairsDark/data/raw/dwarfGroups_Hill_135.csv")
old135H = pd.read_csv("dwarfPairsHydro/data/raw/dwarfGroups_Hill_135.csv")
old135DM = pd.read_csv("massivePairsDark/data/raw/massiveGroups_Hill_135.csv")
old135HM = pd.read_csv("massivePairsHydro/data/raw/massiveGroups_Hill_135.csv")


In [17]:
print("Old number of groups -- dwarf dark (non-hill):",len(old135DD))

print("Old number of groups -- dwarf dark:",len(old135D))
print("Old number of groups -- dwarf hydro:",len(old135H))
print("Old number of groups -- massive dark:",len(old135DM))
print("Old number of groups -- massive hydro:",len(old135HM))

Old number of groups -- dwarf dark (non-hill): 13083
Old number of groups -- dwarf dark: 0
Old number of groups -- dwarf hydro: 14410
Old number of groups -- massive dark: 1201
Old number of groups -- massive hydro: 1137


In [19]:
new135 = h5py.File("pears/data/groups/Illustris_135.hdf5",'r')


In [73]:
print("In Illustris:")
new135 = h5py.File("pears/data/groups/Illustris_135.hdf5",'r')
for phys in ["dark","hydro"]:
    for size in ["dwarf","massive"]:
        print(f"New number of groups -- {size} {phys}:",len(new135[phys][size]['Group Number']))

print("In TNG:")
new99 = h5py.File("pears/data/groups/TNG_99.hdf5",'r')
for phys in ["dark","hydro"]:
    for size in ["dwarf","massive"]:
        print(f"New number of groups -- {size} {phys}:",len(new99[phys][size]['Group Number']))

In Illustris:
New number of groups -- dwarf dark: 15357
New number of groups -- massive dark: 1366
New number of groups -- dwarf hydro: 16812
New number of groups -- massive hydro: 1254
In TNG:
New number of groups -- dwarf dark: 17860
New number of groups -- massive dark: 1652
New number of groups -- dwarf hydro: 16231
New number of groups -- massive hydro: 1644


In [37]:
new135subs= h5py.File("pears/data/subhalos/Illustris_135.hdf5",'r')
subsam = new135subs['dark']['dwarf']

In [70]:
uniqueGIDs = np.unique(subsam['Group ID'])
n = 0
for uni in uniqueGIDs:
    groupmask = np.array(subsam['Group ID']) == uni
    maxmassmask = (np.array(subsam['Subhalo Max Mass']) > 8) & (32 > np.array(subsam['Subhalo Max Mass']))
    halomassmask = np.array(subsam['Subhalo Mass'])>1 
    combined = groupmask&halomassmask&maxmassmask
    if True in combined:
        n+=1
n    

13083

In [63]:
len(uniqueGIDs)

15357

In [41]:
subsam.keys()

<KeysViewHDF5 ['Group ID', 'Group Mass', 'Group Radius', 'Nsubs', 'Subhalo ID', 'Subhalo Mass', 'Subhalo Max Mass', 'Subhalo Max Mass Snap', 'Subhalo Med Stellar Mass', 'Subhalo Pos', 'Subhalo Stellar Masses', 'Subhalo Vel']>

In [48]:
np.array(subsam['Subhalo Max Mass'])

array([44.805916, 18.81328 , 25.6542  , ...,  9.901133, 13.450951,
       20.763424], dtype=float32)

In [68]:
import numpy as np
import hdf5libPy3 as hdf5lib
import readsubfHDF5Py3 as readsubfHDF5
import h5py
import pandas as pd
import mergerClass
import sys
from subboxCoordinates import subboxCoordinates as subbox
from vectorCorrection import vectorCorrection as vector
# from tidalIndex import tidalIndex
from hillRadius import hillRadius

####################################################################
# This script picks out groups that pass a group mass criterion    #
# such that the total group mass is between 8e10 < Mh < 5e11       #
# and ensures that the group has at least one subhalo that reaches #
# a maximum mass of 8e10 < Mh < 3.2e11 and with a current mass     #
# greater than 1e10.                                               #
# -----------------------------------------------------------------#
# Katie Chamberlain - Dec 2020                                     #
####################################################################

snap = int(135) # reading in the snapshot number

#####################################
# defining the hardcoded paths, etc #
#####################################
baseDir      = '/xdisk/gbesla/mig2020/rsgrps/gbeslastudents/katie/dwarfPairsDark/'
spacingFile  = baseDir +'data/raw/data_spacing.csv'
saveFile     = baseDir + 'data/raw/dwarfGroups_Hill_'+str(snap)+'.csv'
illustrisDir = '/xdisk/gbesla/mig2020/rsgrps/gbeslastudents/Illustris/GroupCatalogsDark/'

########################################################
# defining the cosmology and reading in group catalogs #
########################################################
h = 0.704 # note that h is NOT a function of z, by definition
obj = readsubfHDF5.subfind_catalog(illustrisDir, snap, grpcat=True, keysel=['GroupFirstSub','GroupPos','SubhaloMass','SubhaloPos','SubhaloGrNr', 'Group_M_TopHat200','Group_R_TopHat200','SubhaloVel','GroupNsubs'])
print('loaded catalog')

#####################################################
# defining the snapshots and redshifts in Illustris #
#####################################################
snapData = pd.read_csv(spacingFile)
snapshots = snapData['Snapshot'].values
scales = snapData['Scale'].values
redshifts = snapData['Redshift'].values

###############################################################
# determining equivalent redshift and scale for this snapshot #
###############################################################
red = redshifts[snapshots == snap][0]
scale = 1/(1+red)

##############################################
# defining each column of the data structure #
##############################################
inds = obj.GroupFirstSub
groupPos = obj.GroupPos
mvirs = obj.Group_M_TopHat200
mvirs = mvirs/h # to get mass in Msun
rvirs = obj.Group_R_TopHat200
rvirs = rvirs * scale # to get rvir in kpc physical
nsubs = obj.GroupNsubs

submass = obj.SubhaloMass
submass = submass/h # to get mass in Msun
subpos = obj.SubhaloPos
subvel = obj.SubhaloVel
subgr = obj.SubhaloGrNr


###################################
#####     Group Mass Cuts     #####
###################################
lowerGroupMass = 8 # 8e10 Msun
upperGroupMass = 50 # 5e11 Msun
groupMask = np.where((mvirs > lowerGroupMass) & (mvirs < upperGroupMass))  # mask groups that pass the first mass cut
groupMasses = mvirs[groupMask] # to get all group masses in Msun instead of Msun/h
groupNumbers = subgr[inds[groupMask]]
firstSubhaloNumbers = inds[groupMask] 

#####################################
#####     Subhalo Mass Cuts     #####
#####################################
lowerSubhaloMass = 8 #8e10 Msun
upperSubhaloMass = 32 #3.2e11 Msun
lowerCurrentMass = 1 # 1e10 Msun

groupsList = list(firstSubhaloNumbers)
groups           = []
groupMasses      = []
groupRadii       = []
isoFlagHill      = []
pertGroupNum     = []
pertMassRatio    = []
pertSep          = []


subhaloMask = mvirs != 0 # to remove all groups with 0 mass

for i in firstSubhaloNumbers: 
    var = min(15, nsubs[subgr[i]]) # allows for groups with <15 subhalos
    # for j in range(var):
    #     subID = i + j # loops through subhalo IDs
    #     currentMass = submass[subID]        
    #     if currentMass >= lowerCurrentMass:
    #         inst = mergerClass.MergerTree(snap, subID)
    #         maxMass = inst.maxMass()[0]/h
    #         if maxMass >= lowerSubhaloMass and maxMass <= upperSubhaloMass:
    #             groups.append(subgr[i])
    #             break

    # testing for isolation
    groupNum = subgr[i] # finding the group number
    xLo, xHi, yLo, yHi, zLo, zHi = subbox(groupPos[groupNum]) # in ckpc/h
    xGrps, yGrps, zGrps = groupPos[:,0], groupPos[:,1], groupPos[:,2]
    xPos, yPos, zPos = groupPos[groupNum][0], groupPos[groupNum][1], groupPos[groupNum][2]
    groupMass = mvirs[groupNum]
    groupRadius = rvirs[groupNum]
    groupMasses.append(groupMass) 
    groupRadii.append(groupRadius)   

    if xLo < xHi:
        xMask = np.logical_and(xGrps > xLo, xGrps < xHi)
    else:
        xMask = np.logical_or(xGrps > xLo, xGrps < xHi)

    if yLo < yHi:
        yMask = np.logical_and(yGrps > yLo, yGrps < yHi)
    else:
        yMask = np.logical_or(yGrps > yLo, yGrps < yHi)

    if zLo < zHi:
        zMask = np.logical_and(zGrps > zLo, zGrps < zHi)
    else: 
        zMask = np.logical_or(zGrps > zLo, zGrps < zHi)

    equivMask = np.logical_and(xGrps == xPos, yGrps == yPos) # to remove the group itself
    massMask  = (mvirs > groupMass) # only consider groups with masses higher than the group mass

    fullMask = xMask&yMask&zMask&~equivMask&subhaloMask&massMask

    if (~fullMask).all():
        massMask  = (mvirs > groupMass/10) # only consider groups with masses higher than the group mass
        fullMask = xMask&yMask&zMask&~equivMask&subhaloMask&massMask
        if (~fullMask).all():
            fullMask = xMask&yMask&zMask&~equivMask&subhaloMask



    linear = np.arange(0,len(fullMask),1)
    
print(len(linear))



loaded catalog


KeyboardInterrupt: 